In [6]:
import pandas as pd
from scipy import stats
import numpy as np

from src.vizualization import VibrationPlots, interactive_outlier_plot
from src.statistical import StatisticalAnalysis

In [13]:
df_raw: pd.DataFrame = pd.read_csv(
    'data/processed_data.csv',
    parse_dates=["Time"],
    index_col=[0],
    sep=';'
)

# Drop rows where 'Vibration' column has NaN values
df = df_raw.dropna(subset=['feature']).copy()

threshold = 1500
df_clean = df[df['Corrente motor'] > threshold]
# drop all rows with NaN values
df_clean = df_clean.dropna()

# Add Z-score
# df_clean['z_score'] = stats.zscore(df_clean['feature'])
# df['z_score'] = stats.zscore(df['feature'])

In [15]:
length = len(df_clean)
print(f"Data length: {length}")

Data length: 184600


In [17]:
# count the number of samples in column 'feature' that are greater than 2
def count_outliers(df: pd.DataFrame, column: str):
    return df[df[column] > 1.5].shape[0]

print(count_outliers(df_clean, 'feature'))

# percentage of outliers
def percentage_outliers(df: pd.DataFrame, column: str):
    return df[df[column] > 1.5].shape[0] / df.shape[0] * 100

print(percentage_outliers(df_clean, 'feature'))

12909
6.992957746478873


In [23]:
def create_vibration_windows(df, feature_column='feature', window_size=30, threshold=1.5):
    # Step 1: Identify indices where the feature column value is greater than the threshold
    end_indices = np.where(df[feature_column] > threshold)[0]

    # Step 2: Create windows of the specified size ending at these indices
    windows = []
    used_indices = set()

    for idx in end_indices:
        start_idx = idx - (window_size - 1)  # Calculate the start of the window
        if start_idx < 0:
            continue  # Skip windows that start before the beginning of the DataFrame

        # Check if the window overlaps with any already used indices
        if any(i in used_indices for i in range(start_idx, idx + 1)):
            continue  # Skip overlapping windows

        # Add the window to the list of windows
        windows.append(df.iloc[start_idx:idx + 1])

        # Mark the indices as used
        used_indices.update(range(start_idx, idx + 1))

    return windows

vibration_windows = create_vibration_windows(df_clean)

# Display the created windows
len(vibration_windows)

1499

In [26]:
# Função para marcar janelas críticas no DataFrame original
def mark_critical_windows(df, feature_column='feature', window_size=30, threshold=1.5):
    """
    Marca as janelas críticas no DataFrame original com a label 1.

    Parâmetros:
    - df: DataFrame original com timestamp index.
    - feature_column: Nome da coluna que representa os dados de vibração (padrão: 'feature').
    - window_size: Tamanho da janela em amostras (padrão: 30).
    - threshold: Valor limite para considerar o final da janela como crítico (padrão: 2).

    Retorna:
    - DataFrame original com uma nova coluna 'label' indicando as janelas críticas.
    """
    # Inicializa a coluna 'label' com 0
    df['label'] = 0

    # Identifica os índices onde o valor da coluna 'feature' é maior que o threshold
    end_positions = np.where(df[feature_column] > threshold)[0]

    # Marca as janelas críticas
    used_positions = set()
    for pos in end_positions:
        start_pos = pos - (window_size - 1)
        if start_pos < 0:
            continue  # Ignora janelas que começam antes do início do DataFrame
        if any(p in used_positions for p in range(start_pos, pos + 1)):
            continue  # Evita sobreposição de janelas

        # Marca todas as amostras da janela como críticas (label = 1)
        df.loc[df.index[start_pos:pos + 1], 'label'] = 1

        # Marca os índices como utilizados
        used_positions.update(range(start_pos, pos + 1))

    return df

df_critical = mark_critical_windows(df_clean)
# print number of rows with label 1
print(df_critical[df_critical['label'] == 1].shape[0])

# save df_critical to a new csv file
df_critical.to_csv('data/processed_data_critical.csv', sep=';')

44970


In [5]:
def check_sequential_windows(windows, freq='T'):
    sequential_flags = []

    for window in windows:
        # Calcula a diferença entre os timestamps consecutivos
        time_diffs = window.index.to_series().diff().dropna()

        # Verifica se todas as diferenças são iguais à frequência esperada
        is_sequential = all(diff == pd.Timedelta(freq) for diff in time_diffs)

        # Adiciona o resultado à lista de flags
        sequential_flags.append(is_sequential)

    return sequential_flags

sequential_flags = check_sequential_windows(vibration_windows)

for i, (window, is_sequential) in enumerate(zip(vibration_windows, sequential_flags)):
        print(f"Janela {i + 1}:")
        # print(window)
        print(f"É sequencial? {is_sequential}")
        print("\n")

ValueError: unit abbreviation w/o a number

In [ ]:
df.head()

In [ ]:
# Verify null values
df.isnull().sum()

In [ ]:
plot_motor = VibrationPlots(df_clean, target='feature')
plot_motor.general()
plot_motor.hist_and_box()

In [ ]:
plot = VibrationPlots(df_clean, target='Corrente motor')
plot.general()
plot.hist_and_box()

In [15]:
# Count the number of outliers
def count_outliers(df: pd.DataFrame):
    return df[df['z_score'] > 3].shape[0]

In [ ]:
plot.z_score()
print(f'Número de outliers: {count_outliers(df)}')
print(f'Número de amostras: {df.shape[0]}')
print(f'Porcentagem de outliers: {count_outliers(df) / df.shape[0] * 100:.2f}%')

In [ ]:
interactive_outlier_plot(df, 'feature')

### **Análise Outliers**

---

#### **1. Z-Score**
- A linha vermelha, que mostra o Z-score, apresenta vários picos acima e abaixo de **±3**, o que indica outliers significativos, porém alguns influenciados por momentos normais de processo, como paradas planejadas.
- A porcentagem de outliers é de 3% do total de amostras presentes.

In [ ]:
analysis = StatisticalAnalysis(df_clean, 'feature')
analysis.perform_analysis()

### **Análise Estatística dos Dados de Vibração**

---

#### **1. Estatísticas Descritivas**
- **Contagem (count):** 194.557 observações, um número significativo de dados coletados minuto a minuto ao longo de 6 meses.
- **Média (mean):** **1.1738**. A vibração média gira em torno desse valor.
- **Desvio Padrão (std):** **0.2644**. A dispersão dos dados em torno da média é moderada.
- **Mínimo (min):** **0.0**. Valores zerados podem indicar paradas ou falhas na medição.
- **Percentis (25%, 50%, 75%):**
   - **25%:** 1.0655  
   - **Mediana (50%):** 1.1897  
   - **75%:** 1.3207  
   - A distribuição dos dados está **levemente concentrada acima de 1.0**, o que significa que a maior parte dos valores é relativamente próxima da média.
- **Máximo (max):** **3.1553**. Existem valores extremos que representam picos de vibração.

---

#### **2. Coeficiente de Variação (CV):**
- O CV é calculado como a razão entre o desvio padrão e a média:
  \[
  CV = \frac{\text{Desvio Padrão}}{\text{Média}} = 0.225
  \]
- **Interpretação:**
   - Um coeficiente de variação de **0.225 (22,5%)** indica uma **variabilidade moderada** nos dados em relação à média.
   - Isso sugere que, apesar da presença de outliers e variações, os valores de vibração não têm dispersão extrema.

---

#### **3. Assimetria (Skewness):**
- **Skewness = -1.60** (valor negativo):
   - A distribuição é **negativamente assimétrica**, ou seja, a cauda esquerda da distribuição é mais longa.
   - Isso indica que existem **valores mais baixos** de vibração (próximos a zero), possivelmente associados a:
     - **Paradas do equipamento**.
     - **Reduções temporárias** na operação.
   - A concentração maior de valores está acima de **1.0**, com raros valores baixos (outliers para a esquerda).

---

#### **4. Curtose:**
- **Curtose = 6.04**:
   - A curtose indica que a distribuição é **leptocúrtica**, ou seja, possui **caudas longas e picos mais elevados** do que uma distribuição normal.
   - Isso sugere a presença de **outliers**, tanto na parte inferior (valores próximos de 0) quanto superior (picos de vibração).

---

#### **5. Análise Geral**
- A distribuição dos dados de vibração é **moderadamente concentrada**, com uma **média de 1.17** e **baixa dispersão** (CV de 22,5%).
- **Assimetria negativa** e **alta curtose** sugerem:
   - **Presença de valores zerados**, possivelmente indicando **falhas** ou **paradas** no sistema.
   - Picos de vibração fora do padrão.
- O comportamento geral sugere que, na maior parte do tempo, os valores de vibração oscilam próximos à média, mas há momentos específicos em que ocorrem **quedas abruptas ou aumentos extremos**.

---

#### **Conclusões**
1. **Análise de Outliers:**
   - Identificar os períodos onde os valores se aproximam de zero e correlacioná-los com registros operacionais.
   - Analisar os valores máximos para entender possíveis **sobrecargas** ou condições críticas do equipamento.

2. **Segmentação Temporal:**
   - Dividir os dados por períodos (ex.: diários ou semanais) para entender o comportamento da sazonalidade mais claramente.
   - Comparar horários e dias específicos para correlacionar com picos e quedas.

3. **Modelagem Preditiva:**
   - A alta curtose e a assimetria sugerem que modelos tradicionais (como regressão linear) podem não capturar bem esses padrões.
   - Modelos como **LSTM**, SARIMA ou técnicas baseadas em **detectores de anomalia** (ex.: Isolation Forest) podem ser usados para prever vibrações e detectar comportamentos anormais.

---


In [ ]:
plot.seasonal_decompose()

### **Análise da Decomposição Sazonal (Seasonal Decompose)**

---

#### **1. Gráfico Geral**
O gráfico gerado pelo `seasonal_decompose` foi dividido em quatro componentes principais:

1. **Série Original (feature):**
   - Representa os valores brutos da série temporal de vibração, registrados **minuto a minuto** durante **8 meses**.
   - A série apresenta uma grande quantidade de oscilações e ruído, mas com padrões visíveis de períodos de estabilidade e instabilidade.

2. **Tendência (Trend):**
   - A **tendência** captura o comportamento de longo prazo dos dados.
   - Observações:
     - Há um crescimento gradual na tendência até meados de 2023-11, seguido de períodos de estabilização.
     - Pequenas quedas bruscas indicam **anomalias** ou **falhas temporárias**, possivelmente associadas a paradas ou intervenções no processo.
     - O aumento recente sugere **picos de vibração recorrentes** no final da série, indicando uma possível mudança nas operações do equipamento.

3. **Sazonalidade (Seasonal):**
   - A **componente sazonal** evidencia **oscilações periódicas** nos dados, capturando padrões de curto prazo.
   - Com um **período de 1440 minutos** (equivalente a 1 dia), observa-se:
     - Um **padrão diário consistente**, onde os valores de vibração oscilam de forma **regular**.
     - Algumas alterações no padrão sazonal ao longo do tempo podem indicar variações no comportamento diário do processo.

4. **Resíduos (Residual):**
   - Os **resíduos** representam os valores que **não foram explicados** pela tendência ou sazonalidade.
   - Observações:
     - A presença de **picos positivos e negativos** sugere **anomalias** ou ruídos nos dados.
     - Resíduos mais acentuados podem estar associados a eventos **incomuns**, como falhas de equipamento, paradas inesperadas ou mudanças bruscas no processo.
     - Em vários momentos, os resíduos mostram **clusters de pontos fora do padrão**, indicando que há variações não capturadas pelos componentes anteriores.

---

#### **2. Interpretação**
A decomposição sazonal revelou os seguintes pontos-chave:
1. **Tendência Ascendente com Oscilações:**
   - A tendência geral dos dados sugere um **aumento gradativo** nos valores de vibração ao longo do tempo.
   - Isso pode estar relacionado a mudanças no funcionamento do equipamento, desgaste ou alterações no processo operacional.

2. **Sazonalidade Diária:**
   - Os dados possuem um **comportamento cíclico diário** (período = 1440 minutos), o que indica padrões regulares de vibração ao longo do dia.
   - Qualquer desvio significativo desse padrão sazonal pode ser **sinal de anomalias** ou **mudanças no processo**.

3. **Resíduos e Anomalias:**
   - Os resíduos exibem **variações significativas**, sugerindo que nem toda a variabilidade dos dados foi explicada pela decomposição.
   - Eventos pontuais, como **picos de vibração ou quedas abruptas**, devem ser analisados com cuidado para entender suas causas.

---


In [ ]:
plot.sma_ema()

#### **Principais Observações:**

1. **Tendência Geral:**
    A série original oscila em torno de uma média central próxima a 1.0.
    Tanto a SMA quanto a EMA indicam uma tendência estável no longo prazo, com variações sutis.

2. **Picos e Ruído:**
    Existem picos bem altos (valores > 2.5) em momentos específicos, que podem ser considerados anomalias ou flutuações extremas.
    Há também períodos com muitos valores baixos, sugerindo possíveis interrupções ou falhas no processo.

3. **Diferença entre SMA e EMA:**
    A EMA acompanha as oscilações da série original de forma mais próxima do que a SMA devido ao maior peso nos dados recentes.
    A SMA apresenta uma suavização mais "atrasada" e estável, o que é esperado.

In [ ]:
plot.wavelet_transform()